In [1]:
import os
import json
import torch
import numpy as np
import torch_geometric as pyg

In [2]:
def load_data(data_path, split):
    # load dataset from the disk
    with open(os.path.join(data_path, "metadata.json")) as f:
        metadata = json.load(f)
    with open(os.path.join(data_path, f"{split}_offset.json")) as f:
        offset = json.load(f)
    offset = {int(k): v for k, v in offset.items()}

    particle_type = np.memmap(os.path.join(data_path, f"{split}_particle_type.dat"), dtype=np.int64, mode="r")
    position = np.memmap(os.path.join(data_path, f"{split}_position.dat"), dtype=np.float32, mode="r")

    return metadata, offset, particle_type, position

In [3]:
metadata, offset, particle_type, position = load_data("../datasets/WaterDropSmall/", "test")

In [4]:
metadata

{'bounds': [[0.1, 0.9], [0.1, 0.9]],
 'sequence_length': 1000,
 'default_connectivity_radius': 0.015,
 'dim': 2,
 'dt': 0.0025,
 'vel_mean': [-3.964619574176163e-05, -0.00026272129664401046],
 'vel_std': [0.0013722809722366911, 0.0013119977252142715],
 'acc_mean': [2.602686518497945e-08, 1.0721623948191945e-07],
 'acc_std': [6.742962470925277e-05, 8.700719180424815e-05]}

In [5]:
offset, particle_type.shape, position.shape

({0: {'particle_type': {'offset': 0, 'shape': [295]},
   'position': {'offset': 0, 'shape': [1001, 295, 2]}},
  1: {'particle_type': {'offset': 295, 'shape': [803]},
   'position': {'offset': 590590, 'shape': [1001, 803, 2]}},
  2: {'particle_type': {'offset': 1098, 'shape': [410]},
   'position': {'offset': 2198196, 'shape': [1001, 410, 2]}},
  3: {'particle_type': {'offset': 1508, 'shape': [997]},
   'position': {'offset': 3019016, 'shape': [1001, 997, 2]}},
  4: {'particle_type': {'offset': 2505, 'shape': [271]},
   'position': {'offset': 5015010, 'shape': [1001, 271, 2]}}},
 (2776,),
 (5557552,))

In [71]:
noise_std = 3e-4
window_length = 7
dim = 2
windows = []
for traj in offset.values():
    size = traj["position"]["shape"][1]
    length = traj["position"]["shape"][0] - window_length + 1
    for i in range(length):
        desc = {
            "size": size,
            "type": traj["particle_type"]["offset"],
            "pos": traj["position"]["offset"] + i * size * dim,
        }
        windows.append(desc)

In [72]:
len(windows), windows

(4975,
 [{'size': 295, 'type': 0, 'pos': 0},
  {'size': 295, 'type': 0, 'pos': 590},
  {'size': 295, 'type': 0, 'pos': 1180},
  {'size': 295, 'type': 0, 'pos': 1770},
  {'size': 295, 'type': 0, 'pos': 2360},
  {'size': 295, 'type': 0, 'pos': 2950},
  {'size': 295, 'type': 0, 'pos': 3540},
  {'size': 295, 'type': 0, 'pos': 4130},
  {'size': 295, 'type': 0, 'pos': 4720},
  {'size': 295, 'type': 0, 'pos': 5310},
  {'size': 295, 'type': 0, 'pos': 5900},
  {'size': 295, 'type': 0, 'pos': 6490},
  {'size': 295, 'type': 0, 'pos': 7080},
  {'size': 295, 'type': 0, 'pos': 7670},
  {'size': 295, 'type': 0, 'pos': 8260},
  {'size': 295, 'type': 0, 'pos': 8850},
  {'size': 295, 'type': 0, 'pos': 9440},
  {'size': 295, 'type': 0, 'pos': 10030},
  {'size': 295, 'type': 0, 'pos': 10620},
  {'size': 295, 'type': 0, 'pos': 11210},
  {'size': 295, 'type': 0, 'pos': 11800},
  {'size': 295, 'type': 0, 'pos': 12390},
  {'size': 295, 'type': 0, 'pos': 12980},
  {'size': 295, 'type': 0, 'pos': 13570},
  {'si

In [102]:
def generate_noise(position_seq, noise_std):
    """Generate noise for a trajectory"""
    velocity_seq = position_seq[:, 1:] - position_seq[:, :-1]
    time_steps = velocity_seq.size(1)
    velocity_noise = torch.randn_like(velocity_seq) * (noise_std / time_steps ** 0.5)
    velocity_noise = velocity_noise.cumsum(dim=1)
    position_noise = velocity_noise.cumsum(dim=1)
    position_noise = torch.cat((torch.zeros_like(position_noise)[:, 0:1], position_noise), dim=1)
    return position_noise


def preprocess(particle_type, position_seq, target_position, metadata, noise_std):
    """Preprocess a trajectory and construct the graph"""
    # apply noise to the trajectory
    position_noise = generate_noise(position_seq, noise_std)
    position_seq += position_noise

    # calculate the velocities of particles
    recent_position = position_seq[:, -1]
    velocity_seq = position_seq[:, 1:] - position_seq[:, :-1]

    # construct the graph based on the distances between particles
    n_particle = recent_position.size(0)
    edge_index = pyg.nn.radius_graph(recent_position, metadata["default_connectivity_radius"], loop=True, max_num_neighbors=n_particle)
    
    # node-level features: velocity, distance to the boundary
    boundary = torch.tensor(metadata["bounds"])
    distance_to_lower_boundary = recent_position - boundary[:, 0]
    distance_to_upper_boundary = boundary[:, 1] - recent_position
    distance_to_boundary = torch.cat((distance_to_lower_boundary, distance_to_upper_boundary), dim=-1)
    distance_to_boundary = torch.clip(distance_to_boundary / metadata["default_connectivity_radius"], -1.0, 1.0)

    # edge-level features: displacement, distance
    dim = recent_position.size(-1)
    edge_displacement = (torch.gather(recent_position, dim=0, index=edge_index[0].unsqueeze(-1).expand(-1, dim)) -
                   torch.gather(recent_position, dim=0, index=edge_index[1].unsqueeze(-1).expand(-1, dim)))
    edge_displacement /= metadata["default_connectivity_radius"]
    edge_distance = torch.norm(edge_displacement, dim=-1, keepdim=True)

    # ground truth for training
    if target_position is not None:
        last_velocity = velocity_seq[:, -1]
        next_velocity = target_position + position_noise[:, -1] - recent_position
        acceleration = next_velocity - last_velocity
        acceleration = (acceleration - torch.tensor(metadata["acc_mean"])) / torch.sqrt(torch.tensor(metadata["acc_std"]) ** 2 + noise_std ** 2)
    else:
        acceleration = None

    print(acceleration.shape)
    print(edge_displacement.shape, edge_distance.shape)
    print(velocity_seq.shape, distance_to_boundary.shape)
    print(particle_type.shape)
    # return the graph with features
    graph = pyg.data.Data(
        x=particle_type,
        edge_index=edge_index,
        edge_attr=torch.cat((edge_displacement, edge_distance), dim=-1),
        y=acceleration,
        pos=torch.cat((velocity_seq.reshape(velocity_seq.size(0), -1), distance_to_boundary), dim=-1)
    )
    return graph

In [105]:
def get(idx, particle_type, position, windows):
    # load corresponding data for this time slice
    window = windows[idx]
    size = window["size"]
    particle_type = particle_type[window["type"]: window["type"] + size].copy()
    particle_type = torch.from_numpy(particle_type)
    position_seq = position[window["pos"]: window["pos"] + window_length*size*dim].copy()
    position_seq.resize(window_length, size, dim)
    position_seq = position_seq.transpose(1, 0, 2)
    target_position = position_seq[:, -1]  # reserve 
    position_seq = position_seq[:, :-1]
    target_position = torch.from_numpy(target_position)
    position_seq = torch.from_numpy(position_seq)

    graph = preprocess(particle_type, position_seq, target_position, metadata, noise_std)
    

In [106]:
get(0, particle_type, position, windows)

torch.Size([295, 2])
torch.Size([1829, 2]) torch.Size([1829, 1])
torch.Size([295, 5, 2]) torch.Size([295, 4])
torch.Size([295])
